# GNN モジュールの作成

GNNモデルを作成する際、既存のGNNモジュールを単純に積み重ねるだけでは不十分な場合がある。
例えば、ノードの重要性やエッジの重みを考慮して隣接情報を集約する新しい方法を考案したいときがある。

このチュートリアルでは、以下のことを学ぶ。

- DGLのメッセージパッシングAPIを理解する。
- 自分でGraphSAGE畳み込みモジュールを実装する。


このチュートリアルは、すでに :doc:`<1_introduction>`を理解していることを前提としています。

In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

## メッセージパッシングとGNN

DGLは、[Gilmer et al.](https://arxiv.org/abs/1704.01212)によって提案されたメッセージパッシングニューラルネットワークに触発された*メッセージパッシングパラダイム*に従う。基本的に、彼らは多くのGNNモデルが以下のフレームワークに適合することを示した。

\begin{align}m_{u\to v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\to v}^{(l-1)}\right)\end{align}

\begin{align}m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\to v}^{(l)}\end{align}

\begin{align}h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)\end{align}


ここで、$M^{(l)}$を*メッセージ関数*、$\sum$を*集約関数*、$U^{(l)}$を*更新関数*と呼ぶ。ここでの$\sum$は任意の関数を表すことができ、必ずしも総和である必要はない。



例えば、[GraphSAGE畳み込み (Hamilton et al., 2017)](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)は以下の数式で表される。

\begin{align}h_{\mathcal{N}(v)}^k\leftarrow \text{Average}\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}\end{align}

\begin{align}h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)\end{align}

ここでのメッセージパッシングには、方向性がある。  
一般にノード$u$からノード$v$に送信されるメッセージは、逆方向のノード$v$からノード$u$に送信されるメッセージとは異なる場合がある。

DGLは、:class:`dgl.nn.SAGEConv <dgl.nn.pytorch.SAGEConv>`を介してGraphSAGEをサポートしているが、  
以下に、自分でGraphSAGE畳み込みを実装する方法を示す。

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata["h"] = h
            # update_all でメッセージの集約を行う
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

このコードの中心部分は、:func:`g.update_all <dgl.DGLGraph.update_all>`関数である。
この関数は、隣接ノードの特徴を集めて平均化する。`update_all`関数には3つの概念がある。

* メッセージ関数``fn.copy_u('h', 'm)``は、ノード特徴を名前``'h'``で*メッセージ*としてコピーし、  
  隣接ノードに名前``'m'``で送信する。

* 集約関数は、```fn.mean('m', 'h_N')``` で、名前``'m'``で受信したすべてのメッセージを平均化し、  
  新しいノード特徴``'h_N'``として保存する。

* ``update_all`` は、DGLにすべてのノードとエッジに対してメッセージと集約関数を呼び出すように指示する。

このコードの中心部分は、:func:`g.update_all <dgl.DGLGraph.update_all>`関数である。  
:func:`g.update_all <dgl.DGLGraph.update_all>`関数は、隣接ノードの特徴を集めて平均化する。

その後、自分のGraphSAGE畳み込み層を積み重ねて、多層のGraphSAGEネットワークを作成できる。

In [3]:
class GCNConv(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNConv, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(fn.copy_u("h", "m"), fn.sum("m", "h_N"))
            h_N = g.ndata["h_N"]
            h_total = h + h_N
            return self.linear(h_total)

In [4]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

### 学習ループ   
以下のデータロードと学習ループのコードは、`1_introduction.ipynb` からコピーされています。  

In [5]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.950, val acc: 0.058 (best 0.058), test acc: 0.064 (best 0.064)
In epoch 5, loss: 1.849, val acc: 0.326 (best 0.426), test acc: 0.354 (best 0.422)
In epoch 10, loss: 1.674, val acc: 0.404 (best 0.426), test acc: 0.404 (best 0.422)
In epoch 15, loss: 1.426, val acc: 0.484 (best 0.484), test acc: 0.489 (best 0.489)
In epoch 20, loss: 1.127, val acc: 0.606 (best 0.606), test acc: 0.582 (best 0.582)
In epoch 25, loss: 0.816, val acc: 0.706 (best 0.706), test acc: 0.676 (best 0.676)
In epoch 30, loss: 0.536, val acc: 0.744 (best 0.744), test acc: 0.721 (best 0.721)
In epoch 35, loss: 0.323, val acc: 0.764 (best 0.766), test acc: 0.748 (best 0.746)
In epoch 40, loss: 0.183, val acc: 0.774 (best 0.774), test acc: 0.760 (best 0.760)
In epoch 45, loss: 0.103, val acc: 0.772 (best 0.774), test acc:

## さらにカスタマイズする

DGLでは、``dgl.function``パッケージの下に多くの組み込みのメッセージと集約関数が用意されている。
詳細は、`APIドキュメント <apifunction>`を参照してください。

これらのAPIを使用すると、新しいグラフ畳み込みモジュールを迅速に実装できる。  
例えば、以下は、重み付き平均を使用して隣接ノードの表現を集約する新しい ``SAGEConv`` を実装している。
``edata``はエッジ特徴を保持でき、メッセージパッシングにも使用できる。

In [11]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

このデータセットのグラフはエッジの重みを持っていないため、モデルの ``forward()`` 関数ですべてのエッジの重みを1に手動で割り当てている。
edge weigthを考慮したい場合は、自分でエッジの重みを設定してください。

In [12]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device)) # torch.ones(g.num_edges(), 1).to(g.device) はエッジの重みが1の場合
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.947, val acc: 0.156 (best 0.156), test acc: 0.144 (best 0.144)
In epoch 5, loss: 1.887, val acc: 0.390 (best 0.390), test acc: 0.374 (best 0.374)
In epoch 10, loss: 1.762, val acc: 0.440 (best 0.440), test acc: 0.469 (best 0.469)
In epoch 15, loss: 1.567, val acc: 0.478 (best 0.478), test acc: 0.526 (best 0.526)
In epoch 20, loss: 1.309, val acc: 0.522 (best 0.522), test acc: 0.560 (best 0.560)
In epoch 25, loss: 1.011, val acc: 0.602 (best 0.602), test acc: 0.615 (best 0.615)
In epoch 30, loss: 0.712, val acc: 0.666 (best 0.666), test acc: 0.693 (best 0.693)
In epoch 35, loss: 0.459, val acc: 0.700 (best 0.700), test acc: 0.719 (best 0.719)
In epoch 40, loss: 0.278, val acc: 0.736 (best 0.736), test acc: 0.742 (best 0.742)
In epoch 45, loss: 0.165, val acc: 0.746 (best 0.746), test acc: 0.759 (best 0.759)
In epoch 50, loss: 0.099, val acc: 0.754 (best 0.754), test acc: 0.759 (best 0.759)
In epoch 55, loss: 0.063, val acc: 0.748 (best 0.754), test acc: 0.764 (best 0

## ユーザが定義した関数によるさらなるカスタマイズ
DGLは、ユーザが定義したメッセージ関数と集約関数を使用して、最大限のの表現を可能にしている。  
以下は、``fn.u_mul_e('h', 'w', 'm')`` と同等のユーザが定義したメッセージ関数である。

In [8]:
def u_mul_e_udf(edges):
    return {"m": edges.src["h"] * edges.data["w"]}

``edges``は、すべてのエッジに対して、ソースノード特徴、エッジ特徴、およぼターゲットノード特徴を表す ``src``、``data``、``dst`` の3つのメンバーを持っている。

集約関数も自分で定義可能である。例えば、以下は受信したメッセージを平均化する組み込みの ``fn.mean('m', 'h_N')`` 関数と同等である。

In [9]:
def mean_udf(nodes):
    return {"h_N": nodes.mailbox["m"].mean(1)}

要するに、DGLはノードを入次数でグループ化し、各グループに対してDGLは受信したメッセージを第2次元に沿ってスタックする。その後、第2次元に沿って平均化を行い、メッセージを集約する。

ユーザが定義した関数でメッセージと集約関数をカスタマイズする詳細については、`APIリファレンス <apiudf>`を参照してください。

## カスタムGNNモジュールのベストプラクティス

DGL は、以下の順序でカスタムGNNモジュールを作成することを推奨しています。


- ``dgl.nn`` モジュールを使用する。
- ``dgl.nn.functional`` 関数を使用する。これには、ノードごとに入力エッジに対してソフトマックスを計算するなど、より低レベルの複雑な操作が含まれている。
- 組み込みのメッセージと集約関数を使用して ``update_all`` を使用する。
- ユーザが定義したメッセージまたは集約関数を使用する。 

## What’s next?

-  `Writing Efficient Message Passing
   Code <guide-message-passing-efficient>`.


In [10]:
# Thumbnail credits: Representation Learning on Networks, Jure Leskovec, WWW 2018
# sphinx_gallery_thumbnail_path = '_static/blitz_3_message_passing.png'